## Importing Packages

In [4]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os, sys
import requests
import gzip
from datetime import datetime

## Extracting Data

In [5]:
url = "https://cct-ds-code-challenge-input-data.s3.af-south-1.amazonaws.com/sr.csv.gz"
sr_zip = requests.get(url, allow_redirects=True)
!tar -xvzf sr.csv.gz 

x sr.csv


In [6]:
sr = pd.read_csv ('sr.csv')

In [7]:
url2 = "https://cct-ds-code-challenge-input-data.s3.af-south-1.amazonaws.com/city-hex-polygons-8.geojson"
hex_geo = gpd.read_file(url2)

## Starting timer

In [8]:
startTime = datetime.now()

## Data Transformation

In [9]:
#Data Transformation for Hex
hex = hex_geo[{'index', 'centroid_lat', 'centroid_lon'}]
hex_df = pd.DataFrame(hex)
hex_df['index'] = hex_df['index'].astype(str)
hex_df['centroid_lat'] = hex_df['centroid_lat'].fillna(0).astype(str).str[0:6]
hex_df['centroid_lon'] = hex_df['centroid_lon'].fillna(0).astype(str).str[0:5]
hex_df['hex_point'] = hex_df['centroid_lat'] + hex_df['centroid_lon'].astype(str)

#Data Transformation for Service Requests (SR)
sr_df = pd.DataFrame(sr)
sr_df['Latitude'] = sr_df['Latitude'].fillna(0).astype(str).str[0:6]
sr_df['Longitude'] = sr_df['Longitude'].fillna(0).astype(str).str[0:5]
sr_df['sr_point'] = sr_df['Latitude'] + sr_df['Longitude'].astype(str)

#Merging SR to Hex
new_merge = pd.merge(sr_df,hex_df, how='left', left_on = 'sr_point', right_on = 'hex_point', indicator= True)
rslt_df = new_merge.loc[new_merge['_merge'] != 'both']

#Saving Failed Merges
rslt_df.to_csv('sr_hex_Failed_Merges.csv')

#Dropping unnecessary columns and renaming
sr_hex_drop = new_merge.drop(['centroid_lon', 'centroid_lat', 'hex_point', 'sr_point', '_merge', 'Unnamed: 0'], axis=1)
sr_hex = sr_hex_drop.rename(columns = {'index': 'h3_level8_index'}).fillna(0)
sr_hex.drop_duplicates(subset ='NotificationNumber',
                     keep = 'first', inplace = True)

#Saving Final Merge
sr_hex.to_csv('sr_hex_Shaun_Moloi.csv')

## Preview of Data

In [10]:
sr_hex.head()

,NotificationNumber,NotificationType,CreationDate,CompletionDate,Duration,CodeGroup,Code,Open,Latitude,Longitude,SubCouncil2016,Wards2016,OfficialSuburbs,directorate,department,ModificationTimestamp,CompletionTimestamp,CreationTimestamp,h3_level8_index
0,1011845437,C3,2017-01-01,2017-01-19,17.988889,WATER MANAGEMENT DEVICE,No Water WMD,False,-34.07,18.54,Subcouncil 23,43,STRANDFONTEIN,Water and Waste Services,Water and Sanitation,2017-01-19 18:41:27+02:00,2017-01-19 18:41:26+02:00,2017-01-01 18:57:26+02:00,88ad368893fffff
1,1011845442,C3,2017-01-01,2017-01-04,2.539896,WATER,Leak at Valve,False,-33.97,18.45,Subcouncil 20,59,NEWLANDS,Water and Waste Services,Water and Sanitation,2017-01-04 06:34:25+02:00,2017-01-04 06:34:24+02:00,2017-01-01 17:36:57+02:00,88ad361701fffff
4,1011844888,C3,2017-01-01,2017-01-23,22.490799,ELECTRICITY TECHNICAL COMPLAINTS,Street Lights - Single Light Out,False,0.0,0.0,None,None,None,Energy and Climate Change,Electricity,2017-01-23 13:07:01+02:00,2017-01-23 13:07:01+02:00,2017-01-01 01:20:16+02:00,0
5,1011845011,C3,2017-01-01,2017-01-25,23.931562,WATER,Broken Leading,False,-34.12,18.86,Subcouncil 8,86,LWANDLE,Water and Waste Services,Water and Sanitation,2017-01-25 07:43:37+02:00,2017-01-25 07:43:30+02:00,2017-01-01 09:22:03+02:00,88ad36d55bfffff
6,1011845299,C3,2017-01-01,2017-01-01,0.015567,SEWER,Sewer: Blocked/Overflow,False,0.0,0.0,None,None,None,Water and Waste Services,Water and Sanitation,2017-01-01 14:34:25+02:00,2017-01-01 14:34:20+02:00,2017-01-01 14:11:55+02:00,0


## Ending Timer

In [12]:
print("Script time")
print(datetime.now() - startTime)

Script time
0:01:36.384484
